# Span情報がない平均報酬強化学習（simulatorあり）

参考：
* [Span-Agnostic Optimal Sample Complexity and Oracle Inequalities for Average-Reward RL](https://arxiv.org/abs/2502.11238)

今回はSpanが不要な平均MDPでの学習です．Simulatorを使います．

表記：
* MDP：$M \in \mathcal{M}$
* 最大作用素：$M(x)_s=\max _a x_{s a}$
* ゲイン：$\rho^\pi(s):=\lim _{T \rightarrow \infty} \frac{1}{T} \mathbb{E}_s^\pi\left[\sum_{t=0}^{T-1} R_t\right]$
* バイアス：$h^\pi(s):=\mathrm{C}-\lim _{T \rightarrow \infty} \mathbb{E}_s^\pi\left[\sum_{t=0}^{T-1}\left(R_t-\rho^\pi\left(S_t\right)\right)\right]$
* Blackwell-最適：$V_\gamma^{\pi^*} \geq V_\gamma^\pi, \forall \pi$ が任意の$\gamma \geq \bar{\gamma}$で成立するような$\bar{\gamma}$が存在．
* 遷移確率の推定：$\widehat{P}\left(s^{\prime} \mid s, a\right)=\frac{1}{n} \sum_{i=1}^n \mathbb{I}\left\{S_{s, a}^i=s^{\prime}\right\}$

## $n$が固定のとき

まず，すべての状態行動について，$n$個ずつサンプルする状況を考えましょう．

**アルゴリズムのポイント** 
基本的には平均報酬を割引MDPで近似して解くアプローチです．ここで，割引率の近似を工夫するのがポイントです．
複数の割引率を選択して，それぞれの$\gamma$で最適方策$\widetilde{\pi}_\gamma$を計算します．

そのために，次の割引率の範囲を構築しましょう：
$$
\mathcal{H}:=\left\{\gamma: \text { there exists an integer } k \text { such that } \sqrt{n} \leq \frac{1}{1-\gamma}=2^k \leq n\right\}
$$
例えば$n=10$なら，
$$
\sqrt{10} \leq 4, 5, 6, 7, 8, 9, 10 \leq 10
$$
であり，$\mathcal{H}=\{3/4, 7/8\}$です．

そして，次の関数を評価します：
$$
\widehat{L}(\gamma):=(1-\gamma) \min _s \tilde{V}_\gamma(s)-2 \frac{1-\gamma}{n}-\underbrace{\alpha(\delta, n) \sqrt{\frac{\left\|\tilde{V}_\gamma\right\|_{\text {span }}+\frac{3}{n}+1}{n}}}_{\approx 分散の項}
$$

最後に
$$
\text { policy } \widehat{\pi}:=\tilde{\pi}_{\hat{\gamma}} \text {, gain lower bound } \hat{\rho}:=\max \{\widehat{L}(\widehat{\gamma}), 0\} 1
$$
where
$$
\widehat{\gamma}=\operatorname{argmax}_{\gamma \in \mathcal{H}} \widehat{L}(\gamma)
$$

を計算して終わりです．

**直感**

直感的には，これは[sample variance penalization (SVP)](https://arxiv.org/abs/0907.3740)と呼ばれる手法の応用です．
SVPの詳細は省略しますが，単にBernsteinの上界を小さくするだけですね．
このアルゴリズムでは，SVPに基づいて，
$$
\max _{\pi, \gamma}(1-\gamma) \min _s \widehat{V}_\gamma^\pi(s)-\sqrt{\frac{\left\|\widehat{V}_\gamma^\pi\right\|_{\mathrm{span}}+1}{n}}
$$
を解くことを目指しています．これは最適化がちょっとむずかしいので，変わりに
$$
\max _\gamma(1-\gamma) \min _s \widehat{V}_\gamma^{\star}(s)-\sqrt{\frac{\left\|\widehat{V}_\gamma^{\star}\right\|_{\text {span }}+1}{n}}
$$
を解いています．

---

このアルゴリズムについて，次の定理が成立します：

高確率で，ゲインの下界$\widehat{\rho}$は次を満たす：

$$
\rho^{\hat{\pi}} 
\geq \hat{\rho} 
\geq \rho^{\star}-\widetilde{\mathcal{O}}(1) \sqrt{\frac{\left\|h^{\star}\right\|_{\text {span }}+1}{n}} 1
$$

---

**直感的な証明**

証明のために，次のバウンドを使います：
$$
\begin{aligned}
\left\|\rho^{\tilde{\pi}_\gamma}-\rho^{\star}\right\|_{\infty} & \stackrel{(i)}{\lesssim}(1-\gamma)\left(\left\|h^{\star}\right\|_{\text {span }}+\left\|V_\gamma^{\hat{\pi}_\gamma}-V_\gamma^{\star}\right\|_{\infty}\right) \\
& \stackrel{(i i)}{\lesssim}(1-\gamma)\left(\left\|h^{\star}\right\|_{\text {span }}+\frac{1}{1-\gamma} \sqrt{\frac{\left\|V_\gamma^{\star}\right\|_{\text {span }}+\left\|\widehat{V}_\gamma^{\star}\right\|_{\text {span }}+1}{n}}\right) \\
& \stackrel{(i i i)}{\lesssim}(1-\gamma)\left\|h^{\star}\right\|_{\text {span }}+\sqrt{\frac{\left\|h^{\star}\right\|_{\text {span }}+\left\|\widehat{V}_\gamma^{\star}\right\|_{\text {span }}+1}{n}}
\end{aligned}
$$

ここで，
1. (i)は平均報酬を割引で近似する式，
2. (ii)はTODOですが，スパンでバウンドできることは覚えておきましょう．
3. (iii)は$\left\|V_\gamma^{\star}\right\|_{\text {span }} \lesssim\left\|h^{\star}\right\|_{\text {span }}$を使っています．

さらに，次の不等式を使います：

$$
\left\|\widehat{V}_\gamma^{\star}\right\|_{\text {span }} \lesssim\left\|V_\gamma^{\star}\right\|_{\text {span }}+\left\|\widehat{V}_\gamma^{\star}-V_\gamma^{\star}\right\|_{\infty} \lesssim\left\|h^{\star}\right\|_{\text {span }}+\frac{1}{1-\gamma} \sqrt{\frac{\left\|h^{\star}\right\|_{\text {span }}+\left\|\widehat{V}_\gamma^{\star}\right\|_{\text {span }}+1}{n}} .
$$

上を解くと，
$$
\left\|\widehat{V}_\gamma^{\star}\right\|_{\text {span }} \lesssim\left\|h^{\star}\right\|_{\text {span }}+\frac{1}{(1-\gamma)^2 n}
$$
を得ます．よって，

$$
\left\|\rho^{\tilde{\pi}_\gamma}-\rho^{\star}\right\|_{\infty} \lesssim \underbrace{(1-\gamma)\left\|h^{\star}\right\|_{\text {span }}}_{T_1}+\underbrace{\sqrt{\frac{\left\|h^{\star}\right\|_{\text {span }}+1}{n}}}_{T_2}+\underbrace{\frac{1}{(1-\gamma) n}}_{T_3}
$$

が得られます．

ここで，$T_2'$，$T_3$は観測できますね．
しかし，$T_1$がわかりません．

* $T_1$は割引率について単調減少な関数なので，Lapski's trickを使える可能性がありますが，今回の場合に使えるかはよくわからないっぽい？[Optimal pointwise adaptive methods in nonparametric estimation](https://www.jstor.org/stable/2959043)などを見るといいかも．

$T_1$について解決するために，直接 $\left\|\rho^{\hat{\pi}_\gamma}-\rho^{\star}\right\|_{\infty}$ をバウンドするのではなく，今回は下界を最適化します．すなわち，
$\rho^\pi \geq(1-\gamma) V_\gamma^\pi-(1-\gamma)\left\|V_\gamma^\pi\right\|_{\text {span }} \mathbf{1}$を利用します．

下界を最大化することだけ考えると，実は$T_1$を$(1-\gamma)\left\|V_\gamma^{\hat{\pi}_\gamma}\right\|_{\text {span }}$で近似できます．よって，

$$
\left\|V_\gamma^{\widehat{\pi}_\gamma}-\widehat{V}_\gamma^{\star}\right\|_{\infty} \lesssim \frac{1}{1-\gamma} \sqrt{\frac{\left\|\hat{V}_\gamma^*\right\|_{\text {span }}+1}{n}}
$$
を使って，
$$
\begin{aligned}
\rho^{\hat{\pi}_\gamma} & \geq(1-\gamma) V_\gamma^{\hat{\pi}_\gamma}-(1-\gamma)\left\|V_\gamma^{\hat{\pi}_\gamma}\right\|_{\text {span }} \mathbf{1} \geq(1-\gamma) \widehat{V}_\gamma^{\star}-(1-\gamma)\left(\left\|\widehat{V}_\gamma^{\star}\right\|_{\text {span }}-C\left\|V_\gamma^{\hat{\pi}_\gamma}-\widehat{V}_\gamma^{\star}\right\|_{\infty}\right) \mathbf{1} \\
& \geq(1-\gamma) \widehat{V}_\gamma^{\star}-(1-\gamma)\left\|\widehat{V}_\gamma^{\star}\right\|_{\text {span }} \mathbf{1}-C^{\prime} \sqrt{\frac{\left\|\widehat{V}_\gamma^{\star}\right\|_{\text {span }}+1}{n}} \mathbf{1}
\end{aligned}
$$

が得られます．


## $\varepsilon$最適方策を出すとき

今度は$\varepsilon$-ゲイン最適な方策を出すアルゴリズムを考えます．

まず，
$$
\mathcal{H}:=\left\{\gamma: \text { there exists an integer } k \text { such that } \sqrt{n} \leq \frac{1}{1-\gamma}=2^k \leq n\right\}
$$
を構築します．そして，それぞれの$\gamma \in \mathcal{H}_i$について，

$$
\begin{aligned}
& \widehat{U}_i(\gamma):=(1-\gamma) \max _s \widetilde{V}_{\gamma, i}(s)+5 \frac{1-\gamma}{n_i}+\frac{2 \alpha\left(\delta, n_i\right)^2}{(1-\gamma) n_i}+4 \alpha\left(\delta, n_i\right) \sqrt{\frac{\left\|\tilde{V}_{\gamma, i}\right\|_{\text {span }}+1+\frac{3}{n_i}}{n_i}} \\
& \widehat{L}_i(\gamma):=(1-\gamma) \min _s \widetilde{V}_{\gamma, i}(s)-2 \frac{1-\gamma}{n_i}-\alpha\left(\delta, n_i\right) \sqrt{\frac{\left\|\tilde{V}_{\gamma, i}\right\|_{\text {span }}+\frac{3}{n_i}+1}{n_i}}
\end{aligned}
$$
とします．

最後に$\widehat{\gamma}_i:=\operatorname{argmin}_{\gamma \in \mathcal{H}_i} \widehat{U}_i(\gamma)-\widehat{L}_i(\gamma)$として，もし$\widehat{U}_i\left(\widehat{\gamma}_i\right)-\widehat{L}_i\left(\widehat{\gamma}_i\right) \leq \varepsilon$ならばアルゴリズムを終了します．

ここで，$\widetilde{\pi}_\gamma$と$\widetilde{V}_\gamma$は
$$
\widehat{V}_\gamma^{\tilde{\pi}_\gamma} \geq \widehat{V}_\gamma^{\star}-\frac{1}{n} 1 \text { and }\left\|\widetilde{V}_\gamma-\widehat{V}_\gamma^{\star}\right\|_{\infty} \leq \frac{1}{n} .
$$
を満たすような方策と関数です．（つまりValue iterationを解いてるだけ）

このとき，$\widehat{\pi}:=\widetilde{\pi}_{\hat{\gamma}_i, i}$は$\varepsilon$-ゲイン最適になります．

こっちをちゃんと証明しましょう．

### サンプル効率の導出

#### ゲインと割引の誤差

weakly communicating な環境では，次が成立します．

* $(1-\gamma)\left(\min _s V_\gamma^{\star}(s)\right) \mathbf{1} \leq \rho^{\star} \leq(1-\gamma)\left(\max _s V_\gamma^{\star}(s)\right) \mathbf{1}$
    * つまり，$\left\|\rho^{\star}-(1-\gamma) V_\gamma^{\star}\right\|_{\infty} \leq(1-\gamma)\left\|V_\gamma^{\star}\right\|_{\text {span }}$.
* 任意の方策$\pi$について，$(1-\gamma)\left(\min _s V_\gamma^\pi(s)\right) \mathbf{1} \leq \rho^\pi \leq(1-\gamma)\left(\max _s V_\gamma^\pi(s)\right) \mathbf{1}$,
    * つまり，$\left\|\rho^\pi-(1-\gamma) V_\gamma^\pi\right\|_{\infty} \leq(1-\gamma)\left\|V_\gamma^\pi\right\|_{\mathrm{span}}$
* 結果として，任意の方策$\pi$について，

$$
\rho^\pi \geq \rho^{\star}-(1-\gamma)\left(\left\|V_\gamma^\pi-V_\gamma^{\star}\right\|_{\infty}+\left\|V_\gamma^{\star}\right\|_{\text {span }}\right)
$$

**ポイント**：このような式は$\left\|\rho^{\star}-(1-\gamma) V_\gamma^{\star}\right\|_{\infty} \leq(1-\gamma)\left\|h^{\star}\right\|_{\text {span }}$として[導出されてきました](RL_useful_lemma/RL_AverageReward_lemma.ipynb)が，上から最適spanでバウンドすると不便です（アルゴリズムは$h^\star$を知らないので）．
その点で，今回のアルゴリズムはEmpirical Bernsteinに通じるものがあります．

証明は[RL_useful_lemma/RL_AverageReward_lemma.ipynb](RL_useful_lemma/RL_AverageReward_lemma.ipynb)に移しました．

### 割引価値関数の誤差バウンド

$
\alpha(\tilde{\delta}, \tilde{n})=24 \sqrt{16 \log \left(\frac{24 S A \tilde{n}^5}{\tilde{\delta}}\right)} \log _2\left(\log _2(\tilde{n}+4)\right)
$とします．
このとき，固定された$\delta > 0$について，確率$1-\delta$以上で，任意の$i \geq 1$と$\gamma \in \mathcal{H}_i$で次が成立します：

$$
\left\|V_\gamma^{\pi_\gamma^*}-\widehat{V}_{\gamma, i}^{\pi_\gamma^*}\right\|_{\infty} \leq \frac{\alpha\left(\delta, n_i\right)}{1-\gamma} \sqrt{\frac{\left\|V_\gamma^{\pi_\gamma^*}\right\|_{\text {span }}+1}{n_i}}
$$

さらに，計算中の方策は次を満たします：
$$
\begin{aligned}
\widehat{V}_{\gamma, i}^{\tilde{\pi}_{\gamma, i}} & \geq \widehat{V}_{\gamma, i}^{\star}-\frac{1}{n_i} \mathbf{1} \\
\left\|\widetilde{V}_{\gamma, i}-\widehat{V}_{\gamma, i}^{\star}\right\|_{\infty} & \leq \frac{1}{n_i} \\
\left\|\widehat{V}_{\gamma, i}^{\tilde{\pi}_{\gamma, i}}-V_\gamma^{\tilde{\pi}_{\gamma, i}}\right\|_{\infty} & \leq \frac{\alpha\left(\delta, n_i\right)}{1-\gamma} \sqrt{\frac{\left\|\widehat{V}_{\gamma, i}^{\tilde{\pi}_{\gamma, i}}\right\|_{\text {span }}+1}{n_i}} .
\end{aligned}
$$

**証明** TODOですが，[The Plug-in Approach for Average-Reward and Discounted MDPs: Optimal Sample Complexity Analysis](https://arxiv.org/abs/2410.07616)のTheorem 9を使えば簡単っぽい．